In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Folium installed
Libraries imported.


In [1]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


In [24]:
address = 'Chicago, IL'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

41.8755616 -87.6244212


#### Our target city will be Chicago, IL, although this can be generalized to any location. I will do one example for Italian. Then, I will run the same program on more types of places.

####  We will start with a 1000 meter radius search to discover all types of Italian places.

In [36]:
search_query = 'Italian'
radius = 1000
print(search_query + ' .... OK!')

Italian .... OK!


In [37]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=F4L0VTXABWY2IKJVQ2LBYEBPH1QAG3ACJZYNJ4WTHRK5FI5T&client_secret=MR5RQY3DORJGETEGOBFJTJQAUTDEO20OR5ATNWQ5KJ33ZLWG&ll=41.8755616,-87.6244212&v=20180604&query=Italian&radius=1000&limit=100'

In [38]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ea765ee9388d7001b6e2836'},
 'response': {'venues': [{'id': '4aa986ccf964a520115420e3',
    'name': 'Italian Village',
    'location': {'address': '71 W Monroe St',
     'crossStreet': 'btwn Dearborn St & Clark St',
     'lat': 41.88092612690439,
     'lng': -87.63024961104458,
     'labeledLatLngs': [{'label': 'display',
       'lat': 41.88092612690439,
       'lng': -87.63024961104458},
      {'label': 'entrance', 'lat': 41.880613, 'lng': -87.630122}],
     'distance': 768,
     'postalCode': '60603',
     'cc': 'US',
     'city': 'Chicago',
     'state': 'IL',
     'country': 'United States',
     'formattedAddress': ['71 W Monroe St (btwn Dearborn St & Clark St)',
      'Chicago, IL 60603',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d110941735',
      'name': 'Italian Restaurant',
      'pluralName': 'Italian Restaurants',
      'shortName': 'Italian',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/i

In [39]:
print("We have found: ", len(results["response"]["venues"]))

We have found:  8


### Now, we will go to each of these found places and see if they have a competitor within 100 meters. If yes, they are at a higher risk of going bankrupt during the COVID-19 outbreak.

In [46]:
data = results["response"]["venues"]
for venue in data:
    v_lat = venue["location"]["lat"]
    v_lng = venue["location"]["lng"]
    search_url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, v_lat, v_lng, VERSION, "Italian", 100, LIMIT)
    res = requests.get(search_url).json()
    verdict = "SAFE" if len(res["response"]["venues"]) < 2 else "IN DANGER"
    print(venue["name"] + ": " + verdict)
    

Italian Village: IN DANGER
Villa Fresh Italian Kitchen Asian Chao: SAFE
Ms. B's Italian Ice Cart: SAFE
Luke's Italian Beef: SAFE
Al's Beef: SAFE
La Cantina Enoteca: IN DANGER
Tutto Italiano: SAFE
Fiera Italiana at Daley Plaza: SAFE


### We see the following result above. This shows us which places are more likely to close due to a lack of a competitive advantage. Now, let's run this on a bigger scale.

In [48]:
types = ["Italian", "Mexican", "Tacos", "Coffee", "Pizza"] # just to name a few
for place_type in types:
    print("***********")
    print("Now analyzing: " + place_type)
    print("------------")
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, place_type, radius, LIMIT)
    results = requests.get(url).json()
    data = results["response"]["venues"]
    for venue in data:
        v_lat = venue["location"]["lat"]
        v_lng = venue["location"]["lng"]
        search_url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, v_lat, v_lng, VERSION, place_type, 100, LIMIT)
        res = requests.get(search_url).json()
        verdict = "SAFE" if len(res["response"]["venues"]) < 2 else "IN DANGER" # accounts for 1 competitor (which is themselves)
        print(venue["name"] + ": " + verdict)
    print("***********")


***********
Now analyzing: Italian
------------
Italian Village: IN DANGER
Villa Fresh Italian Kitchen Asian Chao: SAFE
Ms. B's Italian Ice Cart: SAFE
Luke's Italian Beef: SAFE
Al's Beef: SAFE
La Cantina Enoteca: IN DANGER
Tutto Italiano: SAFE
Fiera Italiana at Daley Plaza: SAFE
***********
***********
Now analyzing: Mexican
------------
Chipotle Mexican Grill: SAFE
Chipotle Mexican Grill: SAFE
Qdoba Mexican Grill: SAFE
Qdoba Mexican Grill: SAFE
Chipotle Mexican Grill: SAFE
Mexican Indepence Day Parada: SAFE
Chipotle Mexican Grill: IN DANGER
La Cocina Mexican Grill: SAFE
Chipotle Mexican Grill - Wabash & Roosevelt, South Loop: IN DANGER
Chipotle Mexican Grill: SAFE
Spanglish: SAFE
***********
***********
Now analyzing: Tacos
------------
Flaco's Tacos: SAFE
Twisted Tacos: SAFE
Mexi-Tacos Food Truck: SAFE
Zacatacos: SAFE
***********
***********
Now analyzing: Coffee
------------
Peet's Coffee & Tea: SAFE
Dollop Coffee & Tea: SAFE
Intelligentsia Coffee: IN DANGER
Liberty Coffee and Tea: 

# And there we have it! We have a list of places that are classified as SAFE or IN DANGER of closing based on the surrounding places. 